In [1]:
from dask_yarn import YarnCluster
from dask.distributed import Client, progress

/home/hadoop/miniconda/lib/python3.8/site-packages/dask_yarn/core.py:16: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import (
/home/hadoop/miniconda/lib/python3.8/site-packages/dask_yarn/core.py:16: FutureWarning: parse_timedelta is deprecated and will be removed in a future release. Please use dask.utils.parse_timedelta instead.
  from distributed.utils import (


# Vectorize

In [2]:
cluster = YarnCluster(environment="/home/hadoop/environment.tar.gz",
                      worker_vcores = 1,
                      worker_memory = "8GiB"
                      )

# Scale cluster out to 8 such workers:
cluster.scale(8)

# Connect to the cluster (before proceeding, you should wait for workers to be registered by the dask scheduler, as below):
client = Client(cluster)

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:  tcp://172.31.29.241:8786
distributed.scheduler - INFO -   dashboard at:                    :32791
distributed.scheduler - INFO - Receive client connection: Client-17fa2d79-fc2d-11ed-addf-0a94ec274d93
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register worker <WorkerState 'tcp://172.31.23.192:45927', name: dask.worker_1, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://172.31.23.192:45927
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register worker <WorkerState 'tcp://172.31.29.35:46747', name: dask.worker_4, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://172.31.29.35:46747
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register worker <WorkerState 'tcp://172.31.21.193:4

In [ ]:
import dask.dataframe as dd
df = dd.read_json("s3://amazon-reviews-ml/json/train/dataset_fr_test.json")


/home/hadoop/miniconda/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/hadoop/miniconda/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/hadoop/miniconda/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [ ]:
df['total_review'] = df['review_title']+ df['review_body']

In [ ]:
import dask_ml.feature_extraction.text

vect = dask_ml.feature_extraction.text.HashingVectorizer()
X = vect.fit_transform(df['total_review'])

In [ ]:
X.blocks[0].compute()

<5000x1048576 sparse matrix of type '<class 'numpy.float64'>'
	with 121880 stored elements in Compressed Sparse Row format>

In [ ]:
y = df['stars'].astype(int)

In [ ]:
import numpy as np
import sklearn.linear_model
import sklearn.pipeline
import dask_ml.wrappers

In [ ]:
sgd = sklearn.linear_model.SGDClassifier(
    tol=1e-3
)
clf = dask_ml.wrappers.Incremental(
    sgd, scoring='accuracy', assume_equal_chunks=True
)
pipe = sklearn.pipeline.make_pipeline(vect, clf)

In [ ]:
classes = y.unique().compute()

In [ ]:
pipe.fit(df['review_body'], y,
         incremental__classes=classes)

Pipeline(steps=[('hashingvectorizer', HashingVectorizer()),
                ('incremental',
                 Incremental(estimator=SGDClassifier(), scoring='accuracy'))])

In [ ]:
predictions = pipe.predict(df['review_body'])
predictions

dask.array<_predict, shape=(nan,), dtype=int64, chunksize=(nan,), chunktype=numpy.ndarray>

In [ ]:
dask_ml.metrics.accuracy_score(y, predictions)

0.5892

# Vectorize + Stop Words

In [8]:
import dask.dataframe as dd
from dask_ml.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('stopwords')
df = dd.read_json("s3://amazon-reviews-ml/json/train/dataset_fr_train.json")


[nltk_data] Downloading package stopwords to /home/hadoop/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
df['total_review'] = df['review_title'] + df['review_body']

In [10]:
X = df['total_review']
y = df['stars'].astype(int)

/home/hadoop/miniconda/lib/python3.8/site-packages/dask_ml/model_selection/_split.py:462: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  warnings.warn(


In [13]:
# Define French stop words
stop_words = set(stopwords.words('french'))

def remove_stopwords(text):
    tokenized_text = word_tokenize(text)
    filtered_text = [word for word in tokenized_text if word.lower() not in stop_words]
    return ' '.join(filtered_text)

# Apply the function to clean stop words in the 'total_review' column
df['total_review'] = df['total_review'].map_partitions(lambda series: series.apply(remove_stopwords), meta=('total_review', 'object'))


In [14]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

/home/hadoop/miniconda/lib/python3.8/site-packages/dask_ml/model_selection/_split.py:462: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  warnings.warn(


In [15]:
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train.compute())
X_test_vectorized = vectorizer.transform(X_test.compute())

# Create a logistic regression classifier
classifier = LogisticRegression()

# Train the classifier on the vectorized training data
classifier.fit(X_train_vectorized, y_train.compute())

# Make predictions on the vectorized testing data
y_pred = classifier.predict(X_test_vectorized)

# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test.compute(), y_pred)
print("Accuracy:", accuracy)

/home/hadoop/miniconda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.829733236842762


# NLP, using textbolb to generate sentimental score

In [20]:
import pandas as pd
import dask.dataframe as dd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from scipy.sparse import hstack
import pandas as pd
from textblob import TextBlob
from textblob_fr import PatternTagger, PatternAnalyzer

In [21]:
df = dd.read_json("s3://amazon-reviews-ml/json/train/dataset_fr_train.json")

In [22]:
df['total_review'] = df['review_title'] + df['review_body']

In [23]:
df['blob'] = df['total_review'].apply(lambda x: TextBlob(x, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer()))

/home/hadoop/miniconda/lib/python3.8/site-packages/dask/dataframe/core.py:3608: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('total_review', 'object'))

  warnings.warn(meta_warning(meta))


In [24]:
df['polarity'] = df['blob'].apply(lambda x: x.sentiment[0], meta=('blob', 'object'))
df['subjectivity'] = df['blob'].apply(lambda x: x.sentiment[1], meta=('blob', 'object'))

In [25]:
X = df[['polarity', 'subjectivity']]
y = df['stars'].astype(int)

In [28]:
# Create the logistic regression model
model = LogisticRegression()

# Fit the model on the training data
model.fit(X, y)



LogisticRegression()

In [29]:
df_test = dd.read_json("s3://amazon-reviews-ml/json/test/dataset_fr_test.json")

df_test['total_review'] = df_test['review_title'] + df_test['review_body']
df_test['blob'] = df_test['total_review'].apply(lambda x: TextBlob(x, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer()))

df_test['polarity'] = df_test['blob'].apply(lambda x: x.sentiment[0], meta=('blob', 'object'))
df_test['subjectivity'] = df_test['blob'].apply(lambda x: x.sentiment[1], meta=('blob', 'object'))


X_test = df_test[['polarity', 'subjectivity']]
y_test = df_test['stars'].astype(int)

/home/hadoop/miniconda/lib/python3.8/site-packages/dask/dataframe/core.py:3608: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('total_review', 'object'))

  warnings.warn(meta_warning(meta))


In [30]:

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8906
